In [1]:
import sys
import time
from pathlib import Path
import ssl

sys.path.append(str(Path('../').resolve()))
assert 'python-nostr' in [Path(path).name for path in sys.path]

from nostr.key import PrivateKey, PublicKey
from client import Client

## I'm loading my key from disk

In [2]:
key_file = Path('./private_key.txt') # saving a private key for testing
if key_file.exists():
    with open(key_file, 'r') as f:
        pk_hex = f.read()
        private_key = PrivateKey.from_hex(pk_hex)
    public_key = private_key.public_key
    print(f'Loaded private key from {key_file}')
    print(f'Public key: {public_key.bech32()}')
else:
    private_key = PrivateKey()
    public_key = private_key.public_key
    print(f"Generated new private key")
    print(f"Public key: {public_key.bech32()}")
    pk_hex = private_key.hex()
    with open(key_file, 'w') as f:
        f.write(pk_hex)

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n


In [3]:
# load client with no keys
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE})

# load client with public key only
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, public_key_hex=public_key.hex())
assert client.public_key.hex() == public_key.hex()
assert client.private_key is None

# load client with public key and private key
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, public_key_hex=public_key.hex(), private_key_hex=pk_hex)
assert client.public_key.hex() == public_key.hex()
assert client.private_key.hex() == pk_hex

# load client with private key only
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, private_key_hex=pk_hex)
assert client.public_key.hex() == public_key.hex()
assert client.private_key.hex() == pk_hex


logged in as public key
	bech32: npub1pccwcuz4yxcftva7fse54qzk6ke57u5v2spg23w4z3fgv9srdxdqga0vms
	hex: 0e30ec705521b095b3be4c334a8056d5b34f728c54028545d51452861603699a
logged in as public key
	bech32: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n
	hex: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
logged in as public key
	bech32: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n
	hex: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
logged in as public key
	bech32: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n
	hex: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3


# client
instantiating a basic client and testing connections

In [4]:
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE})

with client:
    print(client.relay_manager.connection_statuses)
    print('exiting client')
print(client.relay_manager.connection_statuses)

logged in as public key
	bech32: npub1dykzhmqlggxurrku4nymt9ddn4se3t7d76qz4rv35c7gsdz4ynfsl0avw5
	hex: 692c2bec1f420dc18edcacc9b595ad9d6198afcdf6802a8d91a63c88345524d3
{'wss://nostr-2.zebedee.cloud': True, 'wss://nostr-relay.lnmarkets.com': True, 'wss://relay.damus.io': True}
exiting client
{'wss://nostr-2.zebedee.cloud': False, 'wss://nostr-relay.lnmarkets.com': False, 'wss://relay.damus.io': False}


# make a simple text input client

we will instantiate it with the proper options first

In [5]:
public_key.hex()

'2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3'

In [6]:
from client import TextInputClient
import ssl

text_client = TextInputClient(ssl_options={'cert_reqs': ssl.CERT_NONE}, allow_duplicates=True, private_key_hex=pk_hex)

logged in as public key
	bech32: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n
	hex: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3


## run it
with a simple `with` context. the `__enter__` method calls the `run` command of the client so we can just call `pass` so that nothing else happens after we exit the `run` method before the we `__exit__` the clients and the connections are closed.

In [7]:
with text_client:
    pass

select a command:
                	0	E(x)it
                	1	publish note
                	2	get last 10 notes by you
                	3	get last 10 from hex of author
                	4	delete an event
                	5	check deletions
                	6	get metadata by hex of user
                	7	check event
                	8	get recommended server
                	9	get contacts
                	10	print relays
                	11	add relay
                
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id: ff348a6c240f607bcbd4dfe9f134ff5b34115457ad833116bd2fc3084f1d826d
url: wss://nostr-2.zebedee.cloud
	hello?
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id: 8ef29ed0e5d191bc9d5b751f4eccd20f1b13940df886527487245f6cfe9bc13d
url: wss://nostr-2.zebedee.cloud
	hello, world :)
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id: af7dbd564948e9e7ef7a481041c7befedb5ec00b346ea033cd13db9201

/Users/ryanarmstrong/Documents/01_Programming/GitHub/python-nostr/example_client/client.py:101: UserWarning: could not connect to relay.nostr.ch... removing relay.
  warnings.warn(


connection status: {'wss://nostr-2.zebedee.cloud': True, 'wss://nostr-relay.lnmarkets.com': True, 'wss://relay.damus.io': True}
select a command:
                	0	E(x)it
                	1	publish note
                	2	get last 10 notes by you
                	3	get last 10 from hex of author
                	4	delete an event
                	5	check deletions
                	6	get metadata by hex of user
                	7	check event
                	8	get recommended server
                	9	get contacts
                	10	print relays
                	11	add relay
                
connection status: {'wss://nostr-2.zebedee.cloud': True, 'wss://nostr-relay.lnmarkets.com': True, 'wss://relay.damus.io': True, 'wss://relay.nostr.ch': True}
select a command:
                	0	E(x)it
                	1	publish note
                	2	get last 10 notes by you
                	3	get last 10 from hex of author
                	4	delete an event
                	5	check deletions
  

In [8]:
test = text_client.message_store

## note that we can see duplicate events across relays now!

In [9]:
test

{'wss://nostr-2.zebedee.cloud:8ef29ed0e5d191bc9d5b751f4eccd20f1b13940df886527487245f6cfe9bc13d': <nostr.message_pool.EventMessage at 0x10c52bdc0>,
 'wss://nostr-2.zebedee.cloud:af7dbd564948e9e7ef7a481041c7befedb5ec00b346ea033cd13db9201ff1b77': <nostr.message_pool.EventMessage at 0x10c52bd00>,
 'wss://nostr-2.zebedee.cloud:27cf065883f63084af15bdfdab9e24d68126ca0ec1522675b47c05781e005888': <nostr.message_pool.EventMessage at 0x10c529a20>,
 'wss://nostr-2.zebedee.cloud:b6b21ee8f696998d5f4f0a62cde449edab46353ba8c91ceeb5c9a15a3b895b85': <nostr.message_pool.EventMessage at 0x10c52bca0>,
 'wss://nostr-2.zebedee.cloud:4d1c3b10f2e1f7a5fd655d323f2b4cade53284b9765a0ef15cdce6aec0b6fc88': <nostr.message_pool.EventMessage at 0x10c529c30>,
 'wss://nostr-2.zebedee.cloud:260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb': <nostr.message_pool.EventMessage at 0x10c52bcd0>,
 'wss://nostr-2.zebedee.cloud:c84dde640953d1fba80ce80a58c489b6920d92fa3007875e31824d47a4c1b76d': <nostr.message_pool.E